In [ ]:
import re
import string
import urllib
from datetime import datetime

import benchlingapi
import Bio.Restriction as Restriction
import pygsheets
import requests_html
import toml
from Bio.Seq import Seq
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.golden_gate as golden_gate
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()
part_sheet = gc.open_by_key(col["parts"]).worksheet()

# 3G/JUMP/Densmore

In [ ]:
threeg_kit = "https://www.addgene.org/1000000161/"
marionette_kit = "https://www.addgene.org/1000000137/"
jump_plasmids = [
    "https://www.addgene.org/126956/",
    "https://www.addgene.org/126959/",
    "https://www.addgene.org/126960/",
    "https://www.addgene.org/126961/",
    "https://www.addgene.org/126962/",
    "https://www.addgene.org/126963/",
    "https://www.addgene.org/126964/",
    "https://www.addgene.org/126965/",
    "https://www.addgene.org/126966/",
    "https://www.addgene.org/126967/",
    "https://www.addgene.org/126973/",
    "https://www.addgene.org/126974/",
    "https://www.addgene.org/126975/",
    "https://www.addgene.org/126976/",
    "https://www.addgene.org/126991/",
    "https://www.addgene.org/126996/",
    "https://www.addgene.org/127015/",
    "https://www.addgene.org/127047/",
    "https://www.addgene.org/127051/",
    "https://www.addgene.org/127025/",
    "https://www.addgene.org/127000/",
    "https://www.addgene.org/126983/",
]
jump_plasmids = sorted(jump_plasmids)
densmore_kit = "https://www.addgene.org/1000000059/"
densmore_wells = [
    "A1",
    "A5",
    "A9",
    "B1",
    "B5",
    "B9",
    "C1",
    "C5",
    "C9",
    "D1",
    "D5",
    "D6",
    "D7",
    "D8",
    "D9",
    "D10",
    "D11",
    "D12",
    "E1",
    "E2",
    "E3",
    "E4",
    "E5",
    "E6",
    "E7",
]

# Densmore renaming

In [ ]:
for sheet in (strain_sheet, plasmid_sheet):
    rows = sheet.get_all_records()
    # sheet.unlink()
    col_idx = list(rows[0].keys()).index("Aliases*") + 1
    for idx, row in enumerate(rows):
        if base_url(densmore_kit) in row["Source*"]:
            new_aliases = re.sub(
                r"([^()]*)\s\(([^()]*)\)([^()]*)", r"\1\3,\2\3", row["Aliases*"]
            )
            sheet.update_value((idx + 2, col_idx), new_aliases, parse=False)
    # sheet.link() # TODO: this gives 500 error, not sure why

# Ingest parts

In [ ]:
drive_service = plasmid_sheet.client.drive.service
plasmid_folder = col["plasmid_maps"]
plasmid_maps = api.google.list_drive(drive_service, root=plasmid_folder)

In [ ]:
def import_threeg_part(plasmid, seq):
    part = plasmid_to_part(plasmid, Restriction.BsaI, seq)
    name = {
        "P18m": "pT7",
        "P33m": "pMutalik_med",
        "P34m": "pMutalik_weak",
        "C31m": "Bxb1",
        "C40m": "random_blank",
        "C71m": "CinR-CIDDHYRTC",
        "C95m": "T7_RNAP",
        "C114m": "Cas9_recoded",
        "UC16m": "gQi_gRNA_BD",
        "UC17m": "gV1_gRNA_BD",
        "UC20m": "gN2_gRNA_BD",
        "UCT1m": "gQi_gRNA_BE",
    }.get(part["Name*"])
    if name is not None:
        part["Name*"] = name
    else:
        part["Name*"] = re.search(
            r"(\S+)(?: (?:RBS|terminator|integrase|fusion|protease))? \(",
            plasmid["Description"],
        ).group(1)
    part["Author*"] = "Richard Murray lab"
    return part


def import_densmore_part(plasmid, seq):
    part = plasmid_to_part(plasmid, Restriction.BsaI, seq)
    part["Author*"] = "Douglas Densmore lab"
    return part


def import_jump_part(plasmid, seq):
    part = plasmid_to_part(plasmid, Restriction.BsaI, seq)
    part["Name*"] = re.sub(r"^pJUMP\d+-", "", plasmid["Names"])
    part["Author*"] = "Marcos Valenzuela-Ortega, Christopher French"
    return part


def _format_seq(seq):
    return str(seq).lower()


def plasmid_to_part(plasmid, enzyme, seq):
    plasmid_id = plasmid["ID*"]
    subseqs = golden_gate.re_digest(seq, enzyme, linear=False)
    seq, overhang1, overhang2 = subseqs[0]
    part = {}
    part["Name*"] = plasmid["Names"].split(",")[-1]
    part["Tags"] = plasmid["Tags"]
    part["Plasmid/Oligos (Cutter)*"] = f"{plasmid_id}/{enzyme.__name__}"
    # part["Author*"] = ""
    part["Date*"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    part["Upstream overhang*"] = _format_seq(overhang1[0])
    part["Downstream overhang*"] = _format_seq(overhang2[0])
    part["Sequence*"] = _format_seq(overhang1[0] + sequence.get_seq(seq) + overhang2[0])
    part["Organism/codon usage*"] = "E. coli"
    part["Description"] = plasmid["Description"]
    # part["Reference"] = plasmid["Source*"]
    return part


part_rules = [
    (
        lambda x: (base_url(threeg_kit) in x["Source*"]) and (x["Names"][0] != "V"),
        import_threeg_part,
    ),
    (lambda x: base_url(densmore_kit) in x["Source*"], import_densmore_part),
    (
        lambda x: (any(base_url(j) in x["Source*"] for j in jump_plasmids))
        and ("(Empty Backbone)" not in x["Description"]),
        import_jump_part,
    ),
]

In [ ]:
rows = plasmid_sheet.get_all_records()

In [ ]:
rows[0]

In [ ]:
parts = []
for row in tqdm(rows):
    matched = None
    for predicate, rule in part_rules:
        if predicate(row):
            matched = True
            seq_file = api.util.regex_key(
                plasmid_maps, f'{row["ID*"]}\\.', check_duplicates=True
            )["id"]
            seq = api.read_sequence(
                drive_service.files()
                .get_media(fileId=seq_file)
                .execute()
                .decode("utf8")
            )
            part = rule(row, seq)
            parts.append(part)
            break
    if not matched:
        name = row["Names"]
        # print(f"skipping {name}")

In [ ]:
# first_row = workflow.get_next_empty_row(part_sheet)
first_row = 2  # we actually want to insert before existing rows

In [ ]:
parts = workflow.insert_parts(part_sheet, parts, first_row)

In [ ]:
parts[0]

# Part cutting

In [ ]:
from Bio import Restriction

In [ ]:
seq1 = get_plib_seq(drive_service, 1)
seq2 = get_plib_seq(drive_service, 82)
seq3 = get_plib_seq(drive_service, 23)
seq4 = get_plib_seq(drive_service, 95)
seq5 = get_plib_seq(drive_service, 110)

In [ ]:
from Bio.Seq import Seq

In [ ]:
to_join = [
    (sequence.reverse_complement(seq1), Restriction.BsaI, "Name1", "promoter"),
    (sequence.reverse_complement(seq2), Restriction.BsaI, "Name2", "RBS"),
    (seq3, Restriction.BsaI, "Name3", "CDS"),
    (seq4, Restriction.BsaI, "Name4", "terminator"),
    (sequence.reverse_complement(seq5), Restriction.BsaI, "Name5", "misc_feature"),
]

assembly = golden_gate.assemble(to_join, linear=False)
assembly

In [ ]:
with open("/Users/jacob/Downloads/test3.gb", "w") as f:
    f.write(assembly.format("gb"))